In [1]:
from pydantic import BaseModel, Field
from typing import Literal



from langchain_core.tools import tool
from langchain_ollama import ChatOllama



from langgraph.graph import MessagesState

In [ ]:
@tool
def get_weather(city: Literal["nyc", "sf"]):
	"""Use this to get weather information."""
	if city == "nyc":
		return "It is cloudy in NYC, with 5 mph winds in the North-East direction and a temperature of 70 degrees"
	elif city == "sf":
		return "It is 75 degrees and sunny in SF, with 3 mph winds in the South-East direction"
	else:
		raise AssertionError("Unknown city")



tools = [get_weather]

In [ ]:
class ChainOfThoughtResponse(BaseModel):
	"""Response to the user with this structured output."""
	user_prompt: str = Field(description="The original prompt provided by the user.")
	thought: str = Field(description="Logical reasoning before deciding the next step.")
	action: str = Field(description="The action to be taken, chosen from available tools {}.".format(", ".join([tool.name for tool in tools])))
	action_input:str = Field(description="The required input to perform the selected action.") 
	observation: str = Field(description="The outcome or response from executing the action.") 
	justification: str = Field(description="The explanation of why the final answer is relevant to the original prompt provided by the user.")

In [ ]:
class State(MessagesState):
	response: ChainOfThoughtResponse

In [ ]:
from langgraph.prebuilt import create_react_agent
model = ChatOllama(model="llama3.2:1b-instruct-fp16", temperature=0.1, num_predict="2048")
graph = create_react_agent(
	name="chatbot_react_agent", 
	model=model, 
	tools=tools_node, 
	response_format=ResponseWithChainOfThought,
	checkpointer=memory, 
	prompt=SystemMessage(content="You are a helpful assistant!"),
)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

inputs = {"messages": [
	SystemMessage("You are helpful assistant, if the tool get error, you can tell that you don't know the answer"), 
	HumanMessage("what is the weather in sf")
]}
for s in graph.stream(inputs, stream_mode="values"):
	message = s["messages"][-1]
	if isinstance(message, tuple):
		print(message)
	else:
		message.pretty_print()

In [ ]:
prompt = [HumanMessage("What is the capital of France?")]

In [ ]:
model.invoke(prompt)

In [ ]:
system_msg = SystemMessage("""You are a helpful assistant that responds to questions with three exclamation marks.""")
human_msg = HumanMessage('What is the capital of France?')
model.invoke([system_msg, human_msg])

In [ ]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate.from_template("""Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

template.invoke({"context": """The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.""",
	"question": "Which model providers offer LLMs?"
})

In [ ]:
completion = model.invoke(prompt)

In [ ]:
from pydantic import BaseModel
class AnswerWithJustification(BaseModel):
	'''An answer to the user's question along with justification for the answer.'''
	answer: str
	'''The answer to the user's question'''
	justification: str
	'''Justification for the answer'''

In [ ]:
structured_llm = model.with_structured_output(AnswerWithJustification)
structured_llm

In [ ]:
structured_llm.invoke("What weighs more, a pound of bricks or a pound of feathers")